In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

In [11]:
df = pd.read_json('https://www.dropbox.com/s/r7r92hprnx6pfl3/joinTablas.json?dl=1')

In [6]:
display(df.head(3))

,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,Facultad,IDUsuario,Year,Signatura,TipoItem,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,Temas,Union
0,Row0,1547050497000,80000005327627,720.98 A71S,720,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,Row1,1547050500000,80000001191496,720.9861 A71,720,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,Row2,1547138763000,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,arquitectura y diseño,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,LIBRO,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,[quimic],quimic


In [7]:
display(df.columns)

Index(['RowID', 'Fecha', 'IDItem', 'NumeroUbicacion', 'Dewey', 'Ubicacion',
       'Llave', 'Programa', 'Facultad', 'IDUsuario', 'Year', 'Signatura',
       'TipoItem', 'Autor', 'Titulo', 'AnioPublicacion', 'DeweyEspecifico',
       'TemaDewey', 'Temas', 'Union'],
      dtype='object')

In [8]:
df["Peso"] = df.apply(lambda row: 1/2**(2021-row.Year), axis=1 )

In [9]:
pesos_usuarios = pd.DataFrame(columns= df["Signatura"].unique(),dtype=float)
tam = len(df["Signatura"].unique())
pesos_usuarios.columns = pesos_usuarios.columns.astype(str)
display(pesos_usuarios)

ids = df["IDUsuario"].unique()
i = 0
for usuario in ids:
  pesos_usuarios.loc[i] = np.zeros((tam,), dtype=int)
  #Para cada usuario traer sus Deweys y sus pesas asociados
  prestamos = df.loc[df["IDUsuario"]==usuario]
  deweys = prestamos[["Signatura","Peso"]]
  result_user = deweys.groupby("Signatura")["Peso"].sum().reset_index(name="Peso")
  # Recorrer los Deweys y Pesos para plasmarlos en una matriz
  #display(result_user)
  for index, row in result_user.iterrows():
    d = row["Signatura"]
    p=row['Peso']
    pesos_usuarios[d][i] = p
  i = i+1

,720,540,512,712,741,814,823,330,863,696,...,267,114,034,673,447,600,653,263,091,586


In [10]:
pesos_usuarios.to_json(r'C:\Users\johan\Desktop\GitKraken\DataOutOfGit\Limpieza\pesos_usuario_x_dewey.json')

In [8]:
pesos_usuarios = pd.read_json('https://www.dropbox.com/s/cx0lvgwla4ujvpe/pesos_usuario_x_dewey.json?dl=1')

In [15]:
pesos_usuarios['IDUsuario'] = df.IDUsuario.unique()
pesos_usuarios.head(3)

,720,540,512,712,741,814,823,330,863,696,...,114,034,673,447,600,653,263,091,586,IDUsuario
0,2.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,c361c772cd0220bb16dcfb2ad803e4903334ab94
1,0.0,1.0,1.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,87b0e5a61ed712ddfaf5d478ad68c87c825997e9
2,0.0,0.0,0.00,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,afe8c17ba351d274fc5671978918f0c93032d88f


In [21]:
copia = pesos_usuarios.copy()
copia = copia.drop(labels = ["IDUsuario"], axis = 1)
sumatoria = copia.sum(axis=1)
pesos_norm = copia.div(sumatoria, axis=0).fillna(0)

In [22]:
print(type(pesos_norm))

<class 'pandas.core.frame.DataFrame'>


In [23]:
pesos_norm_id = pesos_norm.copy()
pesos_norm_id['IDUsuario'] = df.IDUsuario.unique()
display(pesos_norm_id.shape[0])

24407

In [33]:
def get_temas_usuario(id_usuario, num_deweys):
    pesos_usuario = pesos_norm_id.loc[pesos_norm_id.IDUsuario == id_usuario]
    display(pesos_usuario)
    deweys_importantes = []
    df_temp = pesos_usuario.copy()

    for i in range(0, num_deweys):
        maxvalueIndexLabel = df_temp.idxmax(axis = 1)
        deweys_importantes.append(maxvalueIndexLabel.values[0])
        df_temp = df_temp.drop(labels = maxvalueIndexLabel.values, axis = 1)

    df_cluster_interes = df_cluster_interes.filter(items=deweys_importantes)
    display(df_cluster_interes)

In [34]:
get_temas_usuario('c361c772cd0220bb16dcfb2ad803e4903334ab94', 15)

,720,540,512,712,741,814,823,330,863,696,...,114,034,673,447,600,653,263,091,586,IDUsuario
0,0.380952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,c361c772cd0220bb16dcfb2ad803e4903334ab94


TypeError: reduction operation 'argmax' not allowed for this dtype

In [25]:
def eliminar_cero(id_usuario):
    m1 = (pesos_norm_id['IDUsuario'] == id_usuario)
    m2 = (pesos_norm_id[m1] != 0).all()
    return pesos_norm_id.loc[m1,m2]

In [41]:
display(eliminar_cero('c361c772cd0220bb16dcfb2ad803e4903334ab94'))

,720,711,725,726,428,746,155,770,378,589,771,584,582,586,IDUsuario
0,0.380952,0.047619,0.02381,0.095238,0.095238,0.047619,0.047619,0.047619,0.095238,0.02381,0.02381,0.02381,0.02381,0.02381,c361c772cd0220bb16dcfb2ad803e4903334ab94


In [54]:
def clustering(df_pesos, treshold):
    i = 0
    resultados = pd.DataFrame(columns = ['IDUsuario', 'Cluster', 'Pertenencia','Peso'])
    for usuario in df_pesos.IDUsuario.unique():
        usuario_limpio = eliminar_cero(usuario)
        usuario_limpio = usuario_limpio.drop(labels = 'IDUsuario', axis = 1)
        for column in usuario_limpio:
            if usuario_limpio.iloc[0][column] >= treshold:
                peso = pesos_usuarios.loc[pesos_usuarios.IDUsuario == usuario][column]
                #print(peso.values[0])
                resultados.loc[i] = np.array([usuario, column, usuario_limpio.iloc[0][column],peso.values[0]])
                i = i + 1
    return resultados    

In [55]:
pesos_clustering = clustering(pesos_norm_id, 0)


,IDUsuario,Cluster,Pertenencia,Peso
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,720,0.38095238095238093,2.0
1,c361c772cd0220bb16dcfb2ad803e4903334ab94,711,0.047619047619047616,0.25
2,c361c772cd0220bb16dcfb2ad803e4903334ab94,725,0.023809523809523808,0.125
3,c361c772cd0220bb16dcfb2ad803e4903334ab94,726,0.09523809523809523,0.5
4,c361c772cd0220bb16dcfb2ad803e4903334ab94,428,0.09523809523809523,0.5
...,...,...,...,...
125648,2af3b662cce52945aeec54a03152892f15190444,401,0.4,0.25
125649,ea28f5400ec4129b4e3f4a96bc423491043773b5,511,1.0,0.125
125650,ba922689e879575fd992d1d92e7dd194d3efdcc7,302,0.5,0.125
125651,ba922689e879575fd992d1d92e7dd194d3efdcc7,146,0.5,0.125


In [59]:
pesos_clustering.to_json(r'C:\Users\user\Downloads\pesos_clustering.json')

In [3]:
pesos_clustering = pd.read_json('https://www.dropbox.com/s/3clldbh4bv3v4m8/pesos_clustering.json?dl=1')

In [4]:
pesos_clustering.head(3)

,IDUsuario,Cluster,Pertenencia
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,720,0.380952
1,c361c772cd0220bb16dcfb2ad803e4903334ab94,711,0.047619
2,c361c772cd0220bb16dcfb2ad803e4903334ab94,725,0.023810


In [129]:
from sklearn import preprocessing
join = pd.read_json('https://www.dropbox.com/s/i1komhf7u1c4y95/joinTablas.json?dl=1')
display(join.head(3))

,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,Facultad,IDUsuario,Year,Signatura,TipoItem,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,Temas,Union
0,Row0,1547050497000,80000005327627,720.98 A71S,720,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,Row1,1547050500000,80000001191496,720.9861 A71,720,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,Row2,1547138763000,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,arquitectura y diseño,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,LIBRO,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,[quimic],quimic


In [166]:
def recomendacion_unidad(cluster):
    usuarios = pesos_clustering.loc[pesos_clustering.Cluster == cluster]
    usuarios.Peso = usuarios.Peso.astype(float) 
    usuarios.loc[usuarios.Peso > 10, "Peso"] = 10
    #normalizar columna de peso
    max_value = usuarios["Peso"].max()
    min_value = usuarios["Peso"].min()
    usuarios["Peso"] = (usuarios["Peso"]) / (max_value)
   #filtramos de la tabla de join los rows con el dewey
    prestamos = join.loc[join.Signatura == cluster]
    #join
    prestamos_data = pd.DataFrame(data=prestamos)
    usuario_data = pd.DataFrame(data=usuarios)
    join_tablas = pd.merge(prestamos_data, usuario_data, left_on='IDUsuario', right_on='IDUsuario', how='inner')
    materialRank = join_tablas.groupby(["Llave"])["Peso"].sum().reset_index(name="Peso")
    materialRank = materialRank.sort_values(by=["Peso"],ascending=False)
    return materialRank


In [167]:
lista_recomendaciones = recomendacion_unidad("720")


C:\Users\user\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-166-25714b521e92>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [169]:
prestamos_cluster = pesos_clustering.loc[pesos_clustering.Cluster == "720"]
prestamos_cluster = prestamos_cluster.head(10)
prestamos_cluster["Pertenencia"] = prestamos_cluster["Pertenencia"].astype(float) 
for index, row in prestamos_cluster.iterrows():
    num_prestamos = round(row["Pertenencia"] * 0.7 * 10.0)
    print("Recomendaciones para el usuario:", row["IDUsuario"])
    aux = lista_recomendaciones.head(num_prestamos)
    print(aux.Llave)

Recomendaciones para el usuario: c361c772cd0220bb16dcfb2ad803e4903334ab94
119    309945
519    865116
862    694909
Name: Llave, dtype: object
Recomendaciones para el usuario: 6b39c8ff51a36458e3f68233b82c1ce8bd8cc3d3
119    309945
519    865116
Name: Llave, dtype: object
Recomendaciones para el usuario: a8cd77c9b47b2723fbd8013ce467664f93ad73e6
119    309945
519    865116
Name: Llave, dtype: object
Recomendaciones para el usuario: 145f664af916e3bb70e66a166b3e13ce7b70b27b
119    309945
519    865116
Name: Llave, dtype: object
Recomendaciones para el usuario: c4b92bc46c80a9e24b1cece78317ddb49310db2f
119    309945
Name: Llave, dtype: object
Recomendaciones para el usuario: 473ee07187813004c6460a53053d2e07e225c019
119    309945
519    865116
862    694909
248    608629
Name: Llave, dtype: object
Recomendaciones para el usuario: 9b6c260f80f1c37f686c186e24ead793b937ed46
Series([], Name: Llave, dtype: object)
Recomendaciones para el usuario: f0f3dafe2f6c920183863612a4921b329b73d10c
Series([], 